Written by Doggerlas
Jan 11,2022

Step1:Reimplement original C code with Python

In [ ]:
import numpy as np
import random
np.set_printoptions(threshold=np.inf)#Show full matrix
#Initialize the binarized digital image
alpha=np.zeros((10,8,8,),dtype=int)
photo=np.zeros((16,64,),dtype=int)
#0
alpha[0,0,4:]=1
alpha[0,0:4,4]=1
#1
alpha[1,0,4:]=1
alpha[1,0:,4]=1
#2
alpha[2,0,4:]=1
alpha[2,0:,4]=1
alpha[2,7,4:]=1
#3
alpha[3,4:,4]=1
alpha[3,7:,4:]=1
#4
alpha[4,0,0:4]=1
alpha[4,0:4,3]=1
#5
alpha[5,0,0:4]=1
alpha[5,0:,3]=1
#6
alpha[6,0,0:4]=1
alpha[6,0:,3]=1
alpha[6,7,0:4]=1
#7
alpha[7,4:,3]=1
alpha[7,7,0:4]=1
#8
alpha[8,2:6,2]=1
alpha[8,2:6,5]=1
alpha[8,5,2:6]=1
#9
alpha[9,2:6,2]=1
alpha[9,2:6,5]=1
alpha[9,2,2:6]=1
#print(alpha)

def bitblt(bx,by,n):
 #print("origin_coordinates:",bx,by,n)
 for j in range(8):
  for i in range(8):
   photo[by+j,bx+i]=alpha[n,j,i]
 return

def shot(n):
 #Initialize photo
 for i in range(16):
  for j in range(64):
   photo[i][j]=0;
 #Fill photo
 cursor=0
 pos=1000
 for k in range(4):
  cursor+=random.randint(0,8)
  bx=cursor
  by=random.randint(0,8)
  
  bitblt(bx,by,int((n/pos)%10))
  cursor+=8
  pos/=10
 #print('Original_photo')
 #print(photo)
 #Add noise
 for a in range(16):
  for b in range(64):
   if(random.randint(0,100000000)%10==0):
     if(photo[a][b]==0):
      photo[a][b]=1
     else:
      photo[a][b]=0
 #print('Noisy_photo')
 #print(photo)
 return
#Output two samples for watch
for epoch in range(2):
 print("---------------------------------------epoch={0}-------------------------------------------------".format(epoch))
 random_num=random.randint(0,9999)
 print("random_num={0}".format(random_num))
 shot(random_num)

---------------------------------------epoch=0-------------------------------------------------
random_num=7414
---------------------------------------epoch=1-------------------------------------------------
random_num=7170


Step2:Data preprocessing



  train_data:Generate 1200 Noisy_photos with 84 line of code

---


  train_label:Generate 1200 random_num of line 82

---
  val_data:Generate 400 Noisy_photos with 84 line of code

---


 val_label:Generate 400 random_num of line 82

---


test_data:Generate 400 Noisy_photos with 84 line of code

---


  test_label:Generate 400 random_num of line 82

---



In [ ]:
train_label=[]
train_data=[]
test_label=[]
test_data=[]
val_label=[]
val_data=[]

sum=2000 #Total number train:val:teat=6:2;2
num_epochs=10
batch_size=2



for epoch in range(sum):
 if(epoch<(sum*0.6)):
  random_num=random.randint(0,9999)
  train_label.append(random_num)
  shot(random_num)
  train_data.append(photo)
 elif((sum*0.6)<=epoch<(sum*0.8)):
  random_num=random.randint(0,9999)
  val_label.append(random_num)
  shot(random_num)
  val_data.append(photo)
 else:
  random_num=random.randint(0,9999)
  test_label.append(random_num)
  shot(random_num)
  test_data.append(photo)

train_label = np.array(train_label)
train_data = np.array(train_data)
test_label = np.array(test_label)
test_data = np.array(test_data)
val_label = np.array(val_label)
val_data = np.array(val_data)

train_data = np.reshape(train_data,(1200,-1))
test_data = np.reshape(test_data,(400,-1))
val_data = np.reshape(val_data,(400,-1))

print("train_label.shape",train_label.shape)
print("train_data",train_data.shape)
print("test_label",test_label.shape)
print("test_data",test_data.shape)

train_label.shape (1200,)
train_data (1200, 1024)
test_label (400,)
test_data (400, 1024)


Step3:Building a fully connected neural network

In [ ]:
from keras import models
from keras import layers

model=models.Sequential()

model.add(layers.Dense(64,activation='relu',input_shape=(16*64,)))
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(1))

model.compile(optimizer='rmsprop',loss='mse',metrics=['mae'])

history=model.fit(train_data,train_label,
     validation_data=(val_data,val_label),
     epochs=num_epochs,batch_size=batch_size)



Epoch 1/10
600/600 [==============================] - 2s 2ms/step - loss: 21053042.0000 - mae: 3778.0576 - val_loss: 8522704.0000 - val_mae: 2529.8081
Epoch 2/10
600/600 [==============================] - 1s 2ms/step - loss: 8373729.5000 - mae: 2502.0510 - val_loss: 8469798.0000 - val_mae: 2526.1287
Epoch 3/10
600/600 [==============================] - 1s 2ms/step - loss: 8344526.5000 - mae: 2492.4280 - val_loss: 8510301.0000 - val_mae: 2527.7793
Epoch 4/10
600/600 [==============================] - 1s 2ms/step - loss: 8360630.0000 - mae: 2497.9727 - val_loss: 8490084.0000 - val_mae: 2528.1028
Epoch 5/10
600/600 [==============================] - 1s 2ms/step - loss: 8372388.5000 - mae: 2500.1028 - val_loss: 8470320.0000 - val_mae: 2526.4729
Epoch 6/10
600/600 [==============================] - 1s 2ms/step - loss: 8354945.0000 - mae: 2495.8743 - val_loss: 8489826.0000 - val_mae: 2526.8303
Epoch 7/10
600/600 [==============================] - 1s 2ms/step - loss: 8371916.0000 - mae: 2502.

Step4:Calculate the accuracy

In [ ]:
model.summary()

test_mse_score,test_mae_score=model.evaluate(test_data,test_label)

print("test_mse_score:",test_mse_score)
print("test_mae_score:",test_mae_score)

pred_test_label = model.predict(test_data)
print(pred_test_label)
pred_test_label.shape

from sklearn.metrics import r2_score
pred_acc = r2_score(test_label, pred_test_label)
print('pred_acc',pred_acc)




Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                65600     
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 69,825
Trainable params: 69,825
Non-trainable params: 0
_________________________________________________________________
13/13 [==============================] - 0s 2ms/step - loss: 8596383.0000 - mae: 2563.3579
test_mse_score: 8596383.0
test_mae_score: 2563.35791015625
[[5107.9404]
 [5107.9404]
 [5107.9404]
 [5107.9404]
 [5107.9404]
 [5107.9404]
 [5107.9404]
 [5107.9404]
 [5107.9404]
 [5107.9404]
 [5107.9404]
 [5107.9404]
 [5107.940